# Automating a processing chain using Python apps (scripts)
So far, we have run Jupyter Notebooks to go through the steps involved in the creation of a baseline image composite, the downloading and processing of change detection images, the classification into land cover maps, the change detection and verification, and the aggregation of many change maps into one report image file. We have then vectorised the report image to reduce its file size and emailed the reports to registered users as an email attachment.
This notebook shows how we can build an efficient and more automated forest alerts processing chain. We will do this by basically turning the four previous notebooks into Python scripts (also sometimes called Apps). A Python script ends with .py and is executable from within Python.

# Setup: Requirements to use this Notebook

In [1]:
pwd

'Z:\\gy7709\\pyeo\\notebooks'

In [2]:
cd Z:\gy7709\pyeo

Z:\gy7709\pyeo


We did this in the previous notebook step-by-step. Here, we initialie the notebook in one code cell to speed up the process.

In [3]:
import argparse
import configparser
import cProfile
import datetime
from datetime import datetime
from email.message import EmailMessage
import geopandas as gpd
import glob
import json
import os
from osgeo import gdal
import pandas as pd
import numpy as np
import shutil
import smtplib
import sys
import warnings
import zipfile

from pyeo import (
    classification, 
    filesystem_utilities,
    queries_and_downloads, 
    raster_manipulation,
    vectorisation
)
from pyeo.filesystem_utilities import config_to_log
from pyeo.acd_national import (
    acd_initialisation,
    acd_config_to_log,
    acd_roi_tile_intersection
)

gdal.UseExceptions()
warnings.simplefilter("ignore", category=UserWarning)

print("Libraries successfully imported")

Windows or iOS detected; Patching GetVirtualMemArray. Some functions may not respond as expected.


Libraries successfully imported


C:\Users\hb91\.conda\envs\pyeo_env_w1\lib\abc.py:106: SHDeprecationWarning: AWS functionality will remain in the codebase for now, but won't be actively maintained.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


In [4]:
config_path = "Z:\gy7709\pyeo\pyeo_windows_gy7709.ini"

# make_composite.py

This App downloads and preprocesses Sentinel 2 images into a median image composite.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

%run pyeo\apps\change_detection\make_composite.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

In this configuration, all processing parameters will be taken from the .ini file.

%run pyeo\apps\change_detection\make_composite.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [ ]:
%run pyeo\apps\change_detection\make_composite.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

# detect_change.py
This App downloads and preprocesses Sentinel 2 images for change detection and classifies them using a machine learning model. It performs change detection against a baseline median image composite. Generates a report image file and optionally vectorises it if selected in the ini file.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

%run pyeo\apps\change_detection\detect_change.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

In this configuration, all processing parameters will be taken from the .ini file.

%run pyeo\apps\change_detection\detect_change.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [ ]:
%run pyeo\apps\change_detection\detect_change.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

# send_report.py
This App sends out any new vectorised report files (.shp format) to a list of recipients specified in an email list file. Currently only email is implemented. WhatsApp will be added in the future. Options are set in the .ini file and login details in the credentials file. Any newly created shapefiles found will be zipped and emailed out as an attachment. The zipping process ensures that they are not sent out twice.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo\apps\change_detection\send_report.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo\apps\change_detection\send_report.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [15]:
pwd

'Z:\\gy7709\\36NXG'

In [16]:
cd Z:\gy7709\pyeo

Z:\gy7709\pyeo


In [17]:
%run pyeo\apps\change_detection\send_report.py Z:\gy7709\pyeo\pyeo_windows_gy7709.ini

2024-08-15 21:19:07,783: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,783: INFO: ---                 PROCESSING START                        ---
2024-08-15 21:19:07,783: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,795: INFO: conda environment path found: C:\Users\hb91\.conda\envs\pyeo_env
2024-08-15 21:19:07,808: INFO: Config file that controls the processing run: Z:\gy7709\pyeo\pyeo_windows_gy7709.ini
2024-08-15 21:19:07,814: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,819: INFO: ----------------------------
2024-08-15 21:19:07,823: INFO: Contents of the config file:
2024-08-15 21:19:07,828: INFO: ----------------------------
2024-08-15 21:19:07,833: INFO:   run_mode :  watch_period_seconds
2024-08-15 21:19:07,839: INFO:   forest_sentinel :  model
2024-08-15 21:19:07,843: INFO:   environment :  sen2cor_path
2024-08-15 21:19:07,848: INFO:   raster_pro

Z:\gy7709\tilelist.csv already exists. Renaming the old file.
New file name: Z:\gy7709\tilelist_12.csv


2024-08-15 21:19:09,935: INFO: Region of interest processing based on ROI file.
2024-08-15 21:19:09,946: INFO: Saved Sentinel-2 tile ID list: Z:\gy7709\tilelist_12.csv
2024-08-15 21:19:09,948: INFO: 2 Sentinel-2 tiles to process.
2024-08-15 21:19:09,953: INFO: Sending out the latest reports for Sentinel-2 tile: 36NXG
2024-08-15 21:19:09,959: INFO: Z:\gy7709\36NXG
2024-08-15 21:19:11,990: INFO: Redirecting log output to tile log: Z:\gy7709\36NXG\log\36NXG.log
2024-08-15 21:19:12,134: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,146: INFO: ---                 PROCESSING START                        ---
2024-08-15 21:19:12,147: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,149: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,154: INFO: ---  TILE PROCESSING START: 36NXG                          ---
2024-08-15 21:19:12,157: INFO: --------------------------------

SystemExit: 1

In [ ]:
# needs a different environment with snakeviz installed
snakeviz Z:\gy7709\send_report.prof